In [9]:
import praw
import yaml
from yaml.loader import SafeLoader
import pandas as pd

In [10]:
with open("config.yaml") as f:
    config = yaml.load(f, SafeLoader)

In [4]:
# Create a read-only Reddit instance
reddit = praw.Reddit(client_id = config['reddit']['client_id'],
                     client_secret = config['reddit']['client_secret'],
                     redirect_url = 'http://localhost:8080',
                     user_agent = config['reddit']['user_agent'])

In [17]:
def get_top_posts(subreddit_list = "MachineLearning", limit = 1000, time_filter = 'all'):
    reddit = praw.Reddit(client_id = config['reddit']['client_id'],
                         client_secret = config['reddit']['client_secret'],
                         redirect_url = 'http://localhost:8080',
                         user_agent = config['reddit']['user_agent']
                         )

    posts = reddit.subreddit(subreddit_list).top(time_filter=time_filter, limit=limit)

    # Initialize dataframe
    posts_df = []

    for post in posts:
        posts_df.append({'post_id': post.id,
                        'subreddit': post.subreddit,
                        'created_utc': post.created_utc,
                        'selftext': post.selftext,
                        'post_url': post.url,
                        'post_title':post.title,
                        'link_flair_text': post.link_flair_text,
                        'score': post.score,
                        'num_comments': post.num_comments,
                        'upvote_ratio': post.upvote_ratio
                        })
        
    return pd.DataFrame(posts_df)

In [26]:
posts_df = get_top_posts("MachineLearning+artificial+datascience", limit=1500, time_filter='all')
posts_df.to_csv("data/ds_ml_ai_reddit_posts.csv", header=True, index=False)

In [19]:
posts_df.head()

,post_id,subreddit,created_utc,selftext,post_url,post_title,link_flair_text,score,num_comments,upvote_ratio
0,gh1dj9,MachineLearning,1.589117e+09,,https://v.redd.it/v492uoheuxx41,[Project] From books to presentations in 10s w...,Project,7791,186,0.99
1,kuc6tz,MachineLearning,1.610275e+09,,https://v.redd.it/25nxi9ojfha61,[D] A Demo from 1993 of 32-year-old Yann LeCun...,Discussion,5852,133,0.98
2,g7nfvb,MachineLearning,1.587789e+09,,https://v.redd.it/rlmmjm1q5wu41,[R] First Order Motion Model applied to animat...,Research,4752,111,0.97
3,lui92h,MachineLearning,1.614525e+09,,https://v.redd.it/ikd5gjlbi8k61,[N] AI can turn old photos into moving Images ...,News,4687,230,0.97
4,ohxnts,MachineLearning,1.625977e+09,,https://i.redd.it/34sgziebfia71.jpg,[D] This AI reveals how much time politicians ...,Discussion,4571,228,0.96


In [20]:
# teste
submission = reddit.submission("kuc6tz")

submission.comments.replace_more(limit=None)
for comment in submission.comments.list():
    print(comment.body)


The fact that they also had to know the location of the numbers and that the algorithm was robust to scale changes is impressive for 1993

It's not like they just solved MNIST in 1993, it's one step above that
Every data scientist today is truly standing on the shoulders of giants.
awesome to see
TIL audio hasn’t been invented until 1994
And yet websites still think those obfuscated texts are a good test for robots
Anyone know who the other guys at the end are?
Man, these guys were the real engineers.
Actually, he was 32 years old when he pressed the button. He was 33 by the time he got the results back.
Never going to complain about not having a strong enough GPU again. Very cool.
Wonder what was the RAM and computing power of the system.
Many don’t know it, but before it was done such text recognition was considered impossible, just like AGI and other hard problems. I think text recognition in mail was the first successful real world application of AI.
MNIST irl
that was certainly mo

In [27]:
comments_list = []

for post_id in posts_df['post_id']:
    submission = reddit.submission(post_id)

    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        comments_list.append({'post_id': post_id, 'comment':comment.body})

In [29]:
comments_df = pd.DataFrame(comments_list)
comments_df.to_csv('data/ds_ml_ai_reddit_comments.csv', header=True, index=False)